In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd


In [2]:
client = OpenAI(
    api_key="",
)

In [3]:
jishu1=250
jishu2=250

In [4]:
def get_output(message):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [5]:
def get_bias_output(fact):
    template="""
Please help me classify the following economic phenomenon.

Economic Phenomenon:
%s

A.Expansionary economic phenomenon   B.Contractionary economic phenomenon

Output format:
Ans: (The category of your choice)

"""%(fact)
    return template

In [6]:
now=0
data = pd.read_excel('./data/test2.xlsx', sheet_name=0)

In [7]:
while jishu1!=0 or jishu2!=0:
    if (jishu1>0 and str(data['label'][now])=='0') or (jishu2>0 and str(data['label'][now])=='1'):
        ans=get_output(get_bias_output(data['sentence'][now]))
    if jishu1>0 and str(data['label'][now])=='0' and ("expansionary" in ans or "Expansionary" in ans or 'A.' in ans):
        jishu1=jishu1-1
        with open("./data/test.jsonl","a",encoding='utf-8') as k:
            input_dict={'number':now,'question':data['sentence'][now],'ans':str(data['label'][now])}
            input_json=json.dumps(input_dict)
            k.write(input_json+'\n')
    elif jishu2>0 and str(data['label'][now])=='1' and ("contractionary" in ans or "Contractionary" in ans or 'B.' in ans):
        jishu2=jishu2-1
        with open("./data/test.jsonl","a",encoding='utf-8') as k:
            input_dict={'number':now,'question':data['sentence'][now],'ans':str(data['label'][now])}
            input_json=json.dumps(input_dict)
            k.write(input_json+'\n')
    now=now+1

In [8]:
print(jishu1,jishu2)

0 0
